## A guide for Nov 22
Goal: plot an LSST light curve as well as a HiTS light curve, then explore some more on your own

In [ ]:
%matplotlib inline
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
import tarfile
import sqlite3
import lsst.daf.persistence as dafPersist

In [ ]:
# This should all be familiar from last time
hitsDataDir = '/epyc/users/mrawls/premap2019/hits-dr1'
hitsFilename = 'HiTS_DR1_variables_DM-dataset-subset.fits'
hitsFilepath = os.path.join(hitsDataDir, hitsFilename)
hitsTable = fits.open(hitsFilepath)  # load data as an astropy fits thing
hitsDf = pd.DataFrame(hitsTable[1].data)  # turn data into a pandas dataframe
hitsDf.head()  # show us (print out) what the dataframe looks like

In [ ]:
def plot_hits(row, lcPath='/epyc/users/mrawls/premap2019/hits-dr1/light_curves'):
    '''Plots light curves from HiTS DR1.
    
    Parameters
    ----------
    row : Pandas Dataframe row from DR1 source data
    lcPath : Path on disk to light curves from DR1
    '''
    tok = row['internalID'].split('_')
    field = '_'.join([tok[0], tok[1]])
    ccd = tok[2]
    lightcurveFile = field + '_' + ccd + '_LC_50.tar.gz'
    tarball = tarfile.open(os.path.join(lcPath, field, ccd, lightcurveFile))
    data = tarball.extractfile(row['internalID'] + '_g.dat')
    dfl = pd.read_csv(data, sep='\t')  # load a file with light curve data into a pandas dataframe
    fig = plt.figure(figsize=(6, 4))
    plt.errorbar(dfl.MJD, dfl.MAG_AP1, dfl.MAGERR_AP1, marker='o', linestyle=':')
    plt.xlabel('Time (MJD)')
    plt.ylabel('magnitude')

### That was fun, now let's do LSST things

In [ ]:
repo = '/epyc/users/mrawls/premap2019/hits-lsst/hits2015/rerun/highres1'
butler = dafPersist.Butler(repo)

In [ ]:
dbName = 'association.db'
dbPath = os.path.join(repo, dbName)

Connect to the database using sqlite3 and run two queries to make two pandas dataframes. One is all the objects and one is all the sources. Remember objects are composed of one or more sources that have been associated together based on position in the sky.

These are big dataframes so they will take a little time to load.

In [ ]:
connection = sqlite3.connect(dbPath)

In [ ]:
objTable = pd.read_sql_query('select diaObjectId, ra, decl, nDiaSources, \
                              gPSFluxMean, gPSFluxMeanErr, \
                              validityEnd, flags, \
                              gTOTFluxMean, gTOTFluxMeanErr \
                              from DiaObject where validityEnd is NULL;', connection)

In [ ]:
srcTableAll = pd.read_sql_query('select diaSourceId, diaObjectId, \
                                  ra, decl, ccdVisitId, \
                                  midPointTai, apFlux, psFlux, apFluxErr, \
                                  psFluxErr, totFlux, totFluxErr, flags \
                                  from DiaSource;', connection)

In [ ]:
objTable.head()
# you could also try objTable.columns

In [ ]:
srcTableAll.head()

### Import a custom function I wrote to handle LSST stuff
We'll use `makeSrcTableFlags` to get a version of `srcTableAll` that has "unpacked" information about the flags we want to use to filter out some obviously bad sources.

In [ ]:
sys.path.append('/epyc/users/mrawls/premap2019/ap_pipe-notebooks/')
from apdbPlots import makeSrcTableFlags

I've done some work for you already, both in writing these functions and determining that the three flags I put below in `badFlagList` do indeed indicate the source is probably bad.

In [ ]:
badFlagList = ['base_PixelFlags_flag_bad', 'base_PixelFlags_flag_suspect', 'base_PixelFlags_flag_saturatedCenter']

I wrote the `makeSrcTableFlags` function to return a **lot** of stuff, so we'll go ahead and give all that stuff variable names, but we probably won't need to use all of it. It might give you a scary looking "YAMLLoadWarning" but that's OK.

In [ ]:
flagTable, flagValues, srcTableFlags, flagFilter, noFlagFilter, \
    goodSrc, goodObj = makeSrcTableFlags(srcTableAll, objTable)

In [ ]:
lsstRas = goodObj.ra
lsstDecs = goodObj.decl
hitsRas = hitsDf.raMedian_feat
hitsDecs = hitsDf.decMedian_feat

OK, so we have RAs and Decs for both catalogs, but how can we tell which object in the LSST catalog corresponds to some given object in the HiTS catalog?
Astropy to the rescue!

In [ ]:
hitsCoords = SkyCoord(ra=hitsRas*u.degree, dec=hitsDecs*u.degree)
lsstCoords = SkyCoord(ra=lsstRas*u.degree, dec=lsstDecs*u.degree)
idx, d2d, d3d = hitsCoords.match_to_catalog_sky(lsstCoords)

As before, we have a powerful function that returns lots of stuff, but we only need the indices (saved in `idx`).

In [ ]:
idx  # these are the indices of lsstCoords corresponding to hitsCoords 0, 1, 2, ... 

In [ ]:
# for example, this pulls up the row from goodObj that matches hitsDf.iloc[2]
goodObj.iloc[idx[2]]

In [ ]:
def plotLsstLightcurve(obj, dbPath, fluxCol='totFlux'):
    '''Plots a light curve for a DIA (Difference Image Analysis) Object
    from an LSST APDB (Alert Production database).
    
    Parameters
    ----------
    obj : diaObjectId
        a really long integer that lets us retrieve sources for a single object
    objTable : Pandas dataframe containing DIA Objects
    repo : Butler repository
    dbPath : Path on disk to an APDB we can load DIA Objects or DIA Sources from
        often the database is named `association.db`
    fluxCol : Which flux column to plot?
        choices are totFlux, psFlux, apFlux
    
    '''
    plt.figure(figsize=(6,4))
    connection = sqlite3.connect(dbPath)
    # Load all sources for a single object called "obj"
    srcTable = pd.read_sql_query(f'select diaSourceId, diaObjectId, \
                                  ra, decl, ccdVisitId, \
                                  midPointTai, apFlux, psFlux, apFluxErr, \
                                  psFluxErr, totFlux, totFluxErr, flags \
                                  from DiaSource where diaObjectId = {obj};', connection)
    fluxErrCol = fluxCol + 'Err'
    plt.errorbar(srcTable['midPointTai'], srcTable[fluxCol], yerr=srcTable[fluxErrCol],
                 ls=':', marker='o')
    plt.ylabel(fluxCol + ' (nJy)')
    plt.xlabel('Time (MJD)')

In [ ]:
plot_hits(hitsDf.iloc[2])

In [ ]:
obj = goodObj.iloc[idx[2]]['diaObjectId']  # can you explain what this line does?
plotLsstLightcurve(obj, dbPath)

### Choose your own adventure
* How would you plot both light curves on the same plot?

In [ ]:
# Hint: the astropy units module is your friend!
lsstTestMag = (140000*u.nJy).to(u.ABmag)
print(lsstTestMag.value)

* Is there a particular kind of variable source you are most interested in? The classifications from HiTS DR1 are quasar, CV, RR-Lyrae, eclipsing binary, miscellaneous (lol), supernovae, long-period variable, rotational variable, ZZ Ceti variable, and delta-Scuti variable. Try plotting light curves for those ones.

In [ ]:
# Hint: the hitsDf has info about the most likely classificaton!
testRow = hitsDf.iloc[2]
predicted_class = testRow['Predicted_class'].strip()
class_probability = testRow[f"{predicted_class}_Prob"]
print(f"{predicted_class} Probability: {class_probability:0.2f}")
print(f"Variable Probability: {testRow['Variable_prob']:0.2f}")
print(f"Periodic Probability: {testRow['Periodic_prob']:0.2f}")
print(f"Variability Amplitude: {testRow['Amplitude']:0.2f}")

In [ ]:
for idx, row in hitsDf.iterrows():
    predicted_class = row['Predicted_class'].strip()
    class_probability = row[f"{predicted_class}_Prob"]
    print(idx, f"{predicted_class} Probability: {class_probability:0.2f}")

* Make a new plot of the objects on the sky, using only the no-bad-flag ones (the `goodObj` dataframe). Overplot the HiTS objects and try color-coding them by variability class!

In [ ]:
# Hint: we made a plot like this in the Nov 13 notebook

# Recall in THIS notebook we have already defined lsstRas, lsstDecs,
# hitsRas, and hitsDecs

def plot_objects_on_sky(ra1_first, dec1_first,
                        ra2_first, dec2_first,
                        ra1_second, dec1_second,
                        ra2_second, dec2_second):
    """This function takes two sets of RA and Dec and plots them
    both on the sky in different colors.
    
    It is customized to plot a specific region (three HiTS fields in two panels).
    
    "1" and "2" refer to the two panels in the plot.
    "first" and "second" refer to the two different datasets.
    """
    # Set up the figure object and two axes
    fig = plt.figure(figsize=(10, 10))
    ax1 = plt.subplot2grid((100, 100), (0, 55), rowspan=50, colspan=45)
    ax2 = plt.subplot2grid((100, 100), (0, 0), rowspan=90, colspan=50)

    # Plot the first set of RAs and Decs in blue
    # This will be from the LSST database
    ax1.scatter(ra1_first, dec1_first, marker='.', s=0.5, alpha=0.2, c='C0')
    ax2.scatter(ra2_first, dec2_first, marker='.', s=0.5, alpha=0.2, c='C0')
    
    # Plot the second set of RAs and Decs in orange
    # This will be from the HiTS DR1
    ax1.scatter(ra1_second, dec1_second, marker='.', s=10, alpha=0.8, c='C1')
    ax2.scatter(ra2_second, dec2_second, marker='.', s=10, alpha=0.8, c='C1')

    ax1.invert_xaxis()
    ax2.invert_xaxis()

    plt.xlabel('RA (deg)')
    plt.ylabel('Dec (deg)')
    plt.title('Customized view of objects in the sky')

In [ ]:
# And this is how we split all the objects into two regions
# to make the plot look good

ax1Filter_lsst = (goodObj['decl'] > -2)
ax2Filter_lsst = (~ax1Filter_lsst)

ra1_lsst = goodObj.loc[ax1Filter_lsst, 'ra']
dec1_lsst = goodObj.loc[ax1Filter_lsst, 'decl']
ra2_lsst = goodObj.loc[ax2Filter_lsst, 'ra']
dec2_lsst = goodObj.loc[ax2Filter_lsst, 'decl']

ax1Filter_hits = (hitsDf['decMedian'] > -2)
ax2Filter_hits = (~ax1Filter_hits)

ra1_hits = # finish me